In [1]:
%reset -f

In [2]:
import pandas as pd
import numpy as np
import os
import sys

In [3]:
# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [4]:
os.getcwd()

'C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data\\notebooks'

In [5]:
os.listdir()

['.ipynb_checkpoints',
 'EDA.ipynb',
 'prep_data.ipynb',
 'prep_data1.ipynb',
 'Visualizations - Bubble chart-Copy1.ipynb',
 'Visualizations - Bubble chart-Copy2.ipynb',
 'Visualizations - Bubble chart.ipynb',
 'Visualizations - Chord diagram.ipynb',
 'Visualizations - Delta on map.ipynb',
 'Visualizations - line chart.ipynb',
 'Visualizations - Main routes.ipynb',
 'Visualizations - Map and Sankey.ipynb',
 'Visualizations - Sales forecast.ipynb']

In [6]:
os.getcwd()

'C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data\\notebooks'

In [7]:
# -----------------------------
# 2. Sample Data Preparation
# -----------------------------
os.chdir("C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data\\data_processed\\Pickel files")
# df_2020_2025 = pd.read_pickle("df_2020_2025.pkl")
df_2024 = pd.read_pickle("df_2024.pkl")

In [8]:
# df = df_2020_2025.copy()  # your 60% sampled dataframe
df = df_2024.copy()  # your 60% sampled dataframe

In [9]:
df.shape

(44303209, 13)

In [12]:
df.head(2)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5748E600F27FCE57,classic_bike,2024-02-04 16:20:38.709,2024-02-04 16:35:35.291,W 20 St & 10 Ave,6306.01,1 Ave & E 18 St,5854.09,40.745686,-74.005141,40.733812,-73.980544,member
1,D24A7073979DC95B,electric_bike,2024-02-07 15:41:36.819,2024-02-07 16:01:12.394,Washington St & Gansevoort St,6039.06,Schermerhorn St & Hoyt St,4479.10,40.739323,-74.008119,40.688626,-73.985191,member


#### 0. Data import

In [13]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from datetime import datetime

#### 1. Chord diagram 

In [14]:
os.getcwd()

'C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data\\data_processed\\Pickel files'

In [15]:
os.chdir("C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data")

In [16]:
os.getcwd()

'C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data'

In [23]:
import pandas as pd
import numpy as np
import json
import colorsys

# def clean_station_names(df):
#     df['start_station_name'] = df['start_station_name'].str.replace('&', 'and')
#     df['end_station_name'] = df['end_station_name'].str.replace('&', 'and')
#     df['start_station_name'] = df['start_station_name'].str.replace(r'[^\w\s-]', '', regex=True)
#     df['end_station_name'] = df['end_station_name'].str.replace(r'[^\w\s-]', '', regex=True)
#     df['start_station_name'] = df['start_station_name'].str[:20]
#     df['end_station_name'] = df['end_station_name'].str[:20]
#     return df

def clean_station_names(df):
    df = df.copy()  # Create a copy to avoid modifying the original DataFrame
    df.loc[:, 'start_station_name'] = df['start_station_name'].str.replace('&', 'and')
    df.loc[:, 'end_station_name'] = df['end_station_name'].str.replace('&', 'and')
    df.loc[:, 'start_station_name'] = df['start_station_name'].str.replace(r'[^\w\s-]', '', regex=True)
    df.loc[:, 'end_station_name'] = df['end_station_name'].str.replace(r'[^\w\s-]', '', regex=True)
    # df.loc[:, 'start_station_name'] = df['start_station_name'].str[:20]
    # df.loc[:, 'end_station_name'] = df['end_station_name'].str[:20]
    return df

    

def get_top_stations(df, n=15):
    start_station_counts = df['start_station_name'].value_counts()
    end_station_counts = df['end_station_name'].value_counts()
    total_counts = pd.concat([start_station_counts, end_station_counts])
    top_stations = total_counts.groupby(level=0).sum().nlargest(n).index.tolist()
    return top_stations

def generate_colors(n):
    """Generate n distinct colors"""
    HSV_tuples = [(x/n, 0.8, 0.9) for x in range(n)]
    return ['#%02x%02x%02x' % tuple(int(x*255) for x in colorsys.hsv_to_rgb(*hsv)) for hsv in HSV_tuples]

def create_chord_data(df, top_stations):
    # Filter for top stations
    df_filtered = df[
        (df['start_station_name'].isin(top_stations)) & 
        (df['end_station_name'].isin(top_stations))
    ]
    
    # Create matrix of flows between stations
    matrix = pd.crosstab(
        df_filtered['start_station_name'],
        df_filtered['end_station_name']
    ).reindex(index=top_stations, columns=top_stations).fillna(0)
    
    return matrix.values.tolist(), top_stations

def create_chord_diagram(matrix, names, output_path):
    # Generate colors for stations
    colors = generate_colors(len(names))
    
    # Create the HTML content
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="utf-8">
        <title>NYC Bike Share Flows</title>
        <script src="https://d3js.org/d3.v7.min.js"></script>
        <style>
            .container {{
                width: 800px;
                height: 500px;
                background-color: rgba(26, 31, 53, 0.0);
            }}
            #chord {{
                width: 800px;
                height: 500px;
                display: flex;
                justify-content: center;
                align-items: center;
            }}
            .tooltip {{
                position: absolute;
                background-color: rgba(26, 31, 53, 0.9);
                color: white;
                padding: 10px;
                border-radius: 5px;
                font-size: 12px;
                pointer-events: none;
            }}
            .station-label {{
                font-family: Arial, sans-serif;
                font-weight: 500;
            }}
        </style>
    </head>
    <body style="margin:0; background-color: transparent;">
        <div class="container">
            <div id="chord"></div>
        </div>
        <script>
            const width = 800;
            const height = 500;
            const radius = Math.min(width, height) * 0.4;  // Increased radius
            const names = {json.dumps(names)};
            const matrix = {json.dumps(matrix)};
            const colors = {json.dumps(colors)};

            // Create the chord diagram
            const svg = d3.select("#chord")
                .append("svg")
                .attr("width", width)
                .attr("height", height)
                .append("g")
                .attr("transform", `translate(${{width/2}},${{height/2}})`);

            // Create a tooltip
            const tooltip = d3.select("body")
                .append("div")
                .attr("class", "tooltip")
                .style("opacity", 0);

            // Create the chord layout
            const chord = d3.chordDirected()
                .padAngle(0.07)  // Increased pad angle for better separation
                .sortSubgroups(d3.descending)
                (matrix);

            // Add the groups
            const group = svg.append("g")
                .selectAll("g")
                .data(chord.groups)
                .join("g");

            // Add the outer arcs
            group.append("path")
                .attr("fill", d => colors[d.index])
                .attr("d", d3.arc()
                    .innerRadius(radius)
                    .outerRadius(radius + 12)  // Increased arc thickness
                );

            // Add station names
            group.append("text")
                .each(d => {{
                    d.angle = (d.startAngle + d.endAngle) / 2;
                    d.name = names[d.index];
                }})
                .attr("dy", ".35em")
                .attr("class", "station-label")
                .attr("transform", d => {{
                    const rotation = (d.angle * 180 / Math.PI - 90);
                    const translation = radius + 30;  // Increased distance for labels
                    return `
                        rotate(${{rotation}})
                        translate(${{translation}})
                        ${{d.angle > Math.PI ? "rotate(180)" : ""}}
                    `;
                }})
                .attr("text-anchor", d => d.angle > Math.PI ? "end" : "start")
                .text(d => d.name)
                .style("font-size", "12px")  // Slightly larger font
                .style("fill", "white")
                .style("font-weight", "bold");

            // Add the chords
            svg.append("g")
                .attr("fill-opacity", 0.8)  // Increased opacity
                .selectAll("path")
                .data(chord)
                .join("path")
                .attr("d", d3.ribbon()
                    .radius(radius)
                )
                .attr("fill", d => colors[d.source.index])
                .attr("stroke", "none")
                .on("mouseover", function(event, d) {{
                    d3.selectAll(".station-label")
                        .style("font-size", text => {{
                            if (text.index === d.source.index || text.index === d.target.index) {{
                                return "14px";  // Larger highlight font
                            }}
                            return "12px";
                        }})
                        .style("fill", text => {{
                            if (text.index === d.source.index || text.index === d.target.index) {{
                                return "#00fff2";
                            }}
                            return "white";
                        }});
                    
                    tooltip.transition()
                        .duration(200)
                        .style("opacity", .9);
                    tooltip.html(`
                        <b>From:</b> ${{names[d.source.index]}}<br/>
                        <b>To:</b> ${{names[d.target.index]}}<br/>
                        <b>Rides:</b> ${{d.source.value.toLocaleString()}}
                    `)
                        .style("left", (event.pageX + 10) + "px")
                        .style("top", (event.pageY - 28) + "px");
                }})
                .on("mouseout", function(d) {{
                    d3.selectAll(".station-label")
                        .style("font-size", "12px")
                        .style("fill", "white");
                    
                    tooltip.transition()
                        .duration(500)
                        .style("opacity", 0);
                }});
        </script>
    </body>
    </html>
    """
    
    # Save the HTML file
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(html_content)

In [24]:
os.getcwd()

'C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data'

In [25]:
os.chdir("C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data")

In [26]:
os.getcwd()

'C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data'

In [21]:
charts_dir = os.path.join('site', 'charts')
os.makedirs(charts_dir, exist_ok=True)

# Clean the data
print("Cleaning data...")
df_clean = df.dropna(subset=['start_station_name', 'end_station_name'])
df_cleaned = clean_station_names(df_clean)

# Get top stations
print("Getting top stations...")
top_stations = get_top_stations(df_cleaned, n=15)

# Create chord diagram data
print("Creating chord diagram data...")
matrix, names = create_chord_data(df_cleaned, top_stations)



Cleaning data...
Getting top stations...
Creating chord diagram data...


In [27]:
# Create and save the visualization
print("Creating chord diagram...")
html_path = os.path.join(charts_dir, 'bike_trips_chord4.html')
create_chord_diagram(matrix, names, html_path)

print(f"Chord diagram saved successfully at {html_path}!")

Creating chord diagram...
Chord diagram saved successfully at site\charts\bike_trips_chord4.html!
